----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content - Streaming Price Cache - Synchronous calls

This notebook demonstrates how to use the Streaming Price interface to cache streaming data from the Refinitiv Data Platform.

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_refinitiv-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _RD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [2]:
import refinitiv.data as rd
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [3]:
rd.open_session()

<refinitiv.data.session.platform.Definition object at 0x15668353e20 {session_name='rdp'}>

## Define and Open a cache for Streaming data
Define a Pricing object from the Streaming data feed - note the **get_stream()** call

In [4]:
# Define our Streaming Price object
streaming = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY='],
    fields=['BID', 'ASK']
).get_stream()
# Open the Stream - Once opened, the library starts caching the updates
streaming.open()
# When you want the current price, get a snapshot
streaming.get_snapshot()

,Instrument,ASK,BID
0,EUR=,1.1223,1.1222
1,GBP=,1.3325,1.3324
2,JPY=,115.26,115.23


#### Call get_snapshot() again, prices may change (assuming instruments are being traded/quoted)

In [5]:
snap = streaming.get_snapshot()
display(snap)

,Instrument,ASK,BID
0,EUR=,1.1224,1.122
1,GBP=,1.3327,1.3323
2,JPY=,115.26,115.25



### Alternative ways of accessing instruments + values

#### Direct Access to fields
We can directly access the cache without the need to call **get_snapshot()**

In [6]:
# Directly access the latest 'BID' price for the EURO
streaming['EUR=']['BID']

1.122

In [7]:
## Access the individual StreamingPrice object for GBP
gbp = streaming['GBP=']
# and then use object to access individual fields
gbp['ASK']

1.3327

In [8]:
# Will be different from above if price has changed
gbp['ASK']

1.3326

#### Iterate on fields

In [9]:
print('JPY=')
for field_name, field_value in streaming['JPY=']:
    print(f"\t{field_name} : {field_value}")

JPY=
	BID : 115.25
	ASK : 115.26


#### Iterate on Streaming instruments and fields

In [10]:
for instrument in streaming:
    print(instrument.name)
    for field_name, field_value in instrument:
        print(f"\t{field_name} : {field_value}")

EUR=
	BID : 1.122
	ASK : 1.1222
GBP=
	BID : 1.3323
	ASK : 1.3327
JPY=
	BID : 115.24
	ASK : 115.27


### Close the Streaming Items when no longer required

In [11]:
streaming.close()

<OpenState.Closed: 3>

### Invalid or un-licensed instruments
What happens if you request using an invalid RIC or an instrument you are not entitled to?
Let's request a mixture of valid and invalid RICs

In [12]:
# Define our Streaming Price object
mixed = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'BADRIC'],
    fields=['BID', 'ASK']
).get_stream()
# Open the Stream - Once opened, the library starts caching the updates
state = mixed.open()
mixed.get_snapshot()

,Instrument,ASK,BID
0,EUR=,1.1224,1.122
1,GBP=,1.3327,1.3323
2,BADRIC,<NA>,<NA>


You can check the Status of any instrument, so lets check the invalid one

In [13]:
mixed['BADRIC'].status

{'ID': 7,
 'Type': 'Status',
 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'BADRIC'},
 'State': {'Stream': 'Closed',
  'Data': 'Suspect',
  'Code': 'NotFound',
  'Text': '**The record could not be found'}}

As you will note, for an invalid instrument we get:   
{'status': <StreamState.Closed: 1>, **'code': 'NotFound'**, 'message': '** The Record could not be found'}   

However, if you are not licensed for the instrument you would see something like:  
{'status': <StreamState.Closed: 1>, **'code': 'NotEntitled'**, 'message': 'A21: DACS User Profile denied access to vendor'}   
**NOTE**: The exact wording of **message** can change over time - therefore,only use the **code** value for any programmatic decision making.

## Close the session

In [14]:
rd.close_session()